In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
# Our Project Dataframe
df = pd.read_csv('product_sales.csv')

# Additional Mapping file U.S. Census Beurau Regions
state_region = pd.read_csv('state_region_mapping.csv')

# Merge
df = df.merge(state_region[['State','Region']].rename(columns={'State':'state','Region':'region'}), on='state',how='left')

In [ ]:
df.head(1)

In [ ]:
df.describe()

In [ ]:
def visualize_null_values(data, bar_width=0.4):
    # Calculate null values for each column
    null_counts = data.isnull().mean() * 100

    # Format the percentages
    null_counts_formatted = null_counts.apply(lambda x: f'{x:.1f}%')

    # Create a bar plot
    fig = go.Figure(go.Bar(
        x=null_counts.index,
        y=null_counts.values,
        text=null_counts_formatted.values,
        textposition='auto',
        marker_color='rgb(55, 83, 109)',
        width=bar_width,
    ))

    # Customize layout
    fig.update_layout(
        title='Percentage of Null Values in Each Column',
        xaxis_title='Columns',
        yaxis_title='Percentage of Null Values',
        showlegend=False,
        template='plotly_white'
    )

    fig.show()


# Call the function to visualize null values
visualize_null_values(df)


In [ ]:
def auto_plot(data):
    # Iterate over columns
    for column in data.columns:
        # Check data type
        if pd.api.types.is_numeric_dtype(data[column]):
            # If numeric, plot histogram
            fig = px.histogram(data, x=column, title=f'Histogram for {column}', labels={'y': 'Count'})
            fig.update_traces(texttemplate='%{y}', textposition='inside')
            fig.show()
        else:
            # If categorical, plot bar chart
            fig = px.bar(data[column].value_counts().reset_index(), x='index', y=column, 
                         title=f'Bar Chart for {column}', labels={'index': 'Categories', column: 'Count'})
            fig.update_traces(texttemplate='%{y}', textposition='inside')
            fig.show()


# Call the function to auto-plot
auto_plot(df.drop(columns=['customer_id']))


### Data Validation

- week:
    - Nothing to clean up

- sales_method:
    - Sames values, different punctuation or simplified text.
    - Bring these into 3 values by:
        - em + call -> Email + Call
        - email -> Email

- nb_sold
    - Nothing to clean up


- revenue:
    - 7.2% of the records have null values.
    - **Decision** Input the median value of this column when it is null.

- years_as_customer
    - Few outliers, max being 63 and average 4.965933.
    - I do not have enough contextual information around this dataset/company to exclude or manipulate.
    - **Decision** given the above, I will not be excluding this.

- nb_cite_visits
    - Nothing to clean up

- state
    - I am bringing in a seperate dataset to map region to states in order to limit this column to a limitted set of values.
    - Focus more on regions could give us a more focused approach to analyzing location data.

- region
    - Nothing to clean up with state, noting to clean up with region.

In [ ]:
# sales_method cleaning
df.loc[df['sales_method'] == 'em + call', 'sales_method'] = 'Email + Call'
df.loc[df['sales_method'] == 'email', 'sales_method'] = 'Email'

# Clean-up revenue
df.loc[df['revenue'].isna(), 'revenue'] = df.dropna(subset=['revenue'])['revenue'].median()

### EDA

- week:
    - First Week saw the largest number of transactions.
        - Dropped in week's 2 and 3, then climbed up again for week's 4 and 5.
        - Regardless, weeks 2-5 saw very similar amounts of transactions.
    - Lowest number of transactions in final week, week 6.
    - Right-swewed.

- sales_method:
    - Email being the most common method, Call + Email being the least common. 
        - Email:        50% of the records
        - Call:         33% of the records
        - Call + Email: 12% of the records

- nb_sold
    - Normally Distrubted, with small peak near the maximum.

- revenue:
    - Multi-modal with multiple peaks (~6) could indicate we are dealing with clients in different business segments (IE Large, Medium, Small, Very-Small, ...) or different products being sold.

- years_as_customer
    - Very Right-Skewed distribution, with visible outlier mentioned above.

- nb_cite_visits
    - Normall Distrubted.

- region
    - The South region being the most transactions United States, and the Northeast region having the least transactions.

In [ ]:
def visualize_null_values(data, bar_width=0.4):
    # Calculate null values for each column
    null_counts = data.isnull().mean() * 100

    # Format the percentages
    null_counts_formatted = null_counts.apply(lambda x: f'{x:.1f}%')

    # Create a bar plot
    fig = go.Figure(go.Bar(
        x=null_counts.index,
        y=null_counts.values,
        text=null_counts_formatted.values,
        textposition='auto',
        marker_color='rgb(55, 83, 109)',
        width=bar_width,
    ))

    # Customize layout
    fig.update_layout(
        title='Percentage of Null Values in Each Column',
        xaxis_title='Columns',
        yaxis_title='Percentage of Null Values',
        showlegend=False,
        template='plotly_white'
    )

    fig.show()


# Call the function to visualize null values
visualize_null_values(df)


In [ ]:
def auto_plot(data):
    # Iterate over columns
    for column in data.columns:
        # Check data type
        if pd.api.types.is_numeric_dtype(data[column]):
            # If numeric, plot histogram
            fig = px.histogram(data, x=column, title=f'Histogram for {column}', labels={'y': 'Count'})
            fig.update_traces(texttemplate='%{y}', textposition='inside')
            fig.show()
        else:
            # If categorical, plot bar chart
            fig = px.bar(data[column].value_counts().reset_index(), x='index', y=column, 
                         title=f'Bar Chart for {column}', labels={'index': 'Categories', column: 'Count'})
            fig.update_traces(texttemplate='%{y}', textposition='inside')
            fig.show()


# Call the function to auto-plot
auto_plot(df.drop(columns=['customer_id']))


In [ ]:
# Selecting only the continuous variables
continuous_vars = df[['nb_site_visits', 'years_as_customer', 'revenue', 'nb_sold']]

# Creating a grid of scatter plots
fig = px.scatter_matrix(continuous_vars, dimensions=list(continuous_vars.columns), 
                        labels={col:col for col in continuous_vars.columns},
                        color_continuous_scale='RdBu', title="Scatterplot Matrix with Correlation Coefficients")

# Customizing layout
fig.update_layout(
    title="Scatterplot Matrix of Continuous Variables",
    width=800,
    height=800,
)

# Display the plots
fig.show()

In [ ]:
# Calculate correlation matrix
corr_matrix = continuous_vars.corr()

# Create a heatmap
fig = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.index,
                   y=corr_matrix.columns,
                   zmin=-1,
                   zmax=1,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation'),
                   text=corr_matrix.round(2)
                ))

# Update layout
fig.update_layout(
    title='Correlation Matrix Heatmap',
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickangle=45),
)
# Display the heatmap
fig.show()


#### Correlation between our continuous variables

- nb_site_visits and years_as_customer
    - from the scatter plot we can see that it resembles a normal distribution.
        - Longer a customer could ininuate that they recieved the average number of visits.
- Revenue and number of units sold has our highest correlation coefficient, which does make sense given the nature of sales.
- Outside of that, the correlaiton between number of site visits and number of units sold has a weak psoitive correlation. 
- I believe the remaining variables have no correlaiton to note with eachother. 

### Questions!

#### How many customers were there for each approach?

- Email: 7466
- Call: 4962
- Email + Call: 2572

In [ ]:
email_method = df[df['sales_method'] == 'Email']['customer_id'].nunique()
call_method = df[df['sales_method'] == 'Call']['customer_id'].nunique()
call_email_method = df[df['sales_method'] == 'Email + Call']['customer_id'].nunique()

In [ ]:
email_method

In [ ]:
call_method

In [ ]:
call_email_method

#### What does the spread of the revenue look like overall? And for each method?

- Overall:
    - Multi-modal descibed in EDA.
    - Center at (descibed by the median) 89.5.
    - Maximum at 238.32
    - Minimum at 32.54

- Email: 
    - Highest number of transactions
    - Fairly normally distributed, with center (decribed by median) 94.275.
    - Maximum at 148.97
    - Minumum at 78.83

- Call:
    - Multi-modal with peaks at 35, 42, 52, 66, and 89.
    - Center at (decribed by median) 49.935.
    - Maximum at 89.5
    - Minimum at 32.54.

- Email + Call
    - Lowest number of transactions
    - Multi-modal with peaks at 87.5, 127.5, 152.5, 185, and 227.5
    - Center at (described by median) 182.135, highest center of all three methods.
    - Maximum at 238.32, highest maximum of all three methods.
    - Minumum at 89.5, highest minum of all three methods.
    


In [ ]:
email_method = df[df['sales_method'] == 'Email']
call_method = df[df['sales_method'] == 'Call']
call_email_method = df[df['sales_method'] == 'Email + Call']

In [ ]:
df['revenue'].min()

In [ ]:
auto_plot(df[['revenue']])

In [ ]:
auto_plot(email_method[['revenue']])

In [ ]:
auto_plot(call_method[['revenue']])

In [ ]:
auto_plot(call_email_method[['revenue']])

#### Was there any difference in revenue over time for each of the methods?

1. Call and Call+Email saw better results as the weeks went on, both with peaking revenue at week 5.
    - Second largest value at week six for Email + Call.

2. The Email method started very strong compared to the other two methods, peaking at week 1. Given the number of transactions this method saw the highest revenue for each week. 

In [ ]:
revnue_group = df.groupby(by=['week', 'sales_method'])['revenue'].sum().reset_index()

In [ ]:
fig = px.bar(revnue_group, x='week', y='revenue', color='sales_method', barmode='group', text=revnue_group['revenue'].apply(lambda x: round(x, 2)), labels={'revenue': 'Revenue'}) 

fig.update_traces(textposition='inside') 

fig.update_layout(title='Revenue by Week and Sales Method',
                  xaxis_title='Week',
                  yaxis_title='Revenue')

fig.show()

### Metric to Monitor
Since the business problem is trying to tackle efficiency, I suggest using the following definition:

Revenue / Number of Site Visits

This metric will measure the amount of revenue brought into the company per site visit. Without knowing the cost of a site visit, this may be not practical. I am assuming this is the highest cost for the sales team during their sales cycle (vs calling, emailing, etc.).

This feature also had the highest correlaiton with our goal, revenue.

In [ ]:
df['revenue_per_visit'] = df['revenue'] / df['nb_site_visits']

In [ ]:
revenue_site_visit_group = df.groupby(by=['week', 'sales_method'])['revenue_per_visit'].mean().reset_index()

In [ ]:
fig = px.bar(revenue_site_visit_group, x='week', y='revenue_per_visit', color='sales_method', barmode='group', text=revenue_site_visit_group['revenue_per_visit'].apply(lambda x: round(x, 2)), labels={'revenue_per_visit': 'Revenue Per Visit'}) 

fig.update_traces(textposition='inside') 

fig.update_layout(title='Revenue Per Visit by Week and Sales Method',
                  xaxis_title='Week',
                  yaxis_title='Average Revenue Per Visit')

fig.show()

##### Results
Judging by our initial values for this metric, we see that Email + Call yields the highest Revenue/Visit across all 6 weeks. Despite revenue needing to ramp up, the investment of both sales strategies (Email + Call) may yield greater returns when visiting potential/current customers.

### Summary

I suggest the business to move forward with the Email + Call sales strategy as it yields the highest Revenue/Visit ratio. If visits are not a high cost part of the sales cycle, I suggest moving forward with just the Email strategy.
